# Get the data in this session

In [ ]:
!git clone https://github.com/katarinaelez/protein-ss-pred.git

fatal: destination path 'protein-ss-pred' already exists and is not an empty directory.


# Declare functions to fetch and format data

In [ ]:
# load packages
%matplotlib inline
import numpy as np
import scipy.stats as stats
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import os.path

import keras
from keras import layers
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
tf.random.set_seed(1)

# functions
def parse_pssm(pssm_filename):
  amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
  num_aas = len(amino_acids)
  sequence = ''
  profile = []

  # fetch protein sequence and frequencies from pssm file
  with open(pssm_filename) as pssm:
    pssm_lines = pssm.readlines()
    for line in pssm_lines[3:-6]:
      profile_line = []
      for n in line.rstrip().split()[22:-2]:
        profile_line.append(float(n)/100)
      profile.append(profile_line)
      sequence += line[6]

  # one-hot encode protein sequence
  encoding = np.zeros((len(sequence), num_aas))
  for i, aa in enumerate(sequence):
    if aa in amino_acids:
      index = amino_acids.index(aa)
      encoding[i, index] = 1
    else: encoding[i, :] = 0.05

  return encoding, profile

def parse_dssp(dssp_filename):
	ss = ''
	with open(dssp_filename) as dssp:
		dssp.readline()
		ss = dssp.readline().rstrip()
	return ss

def fetch_data(window, cv=False, train=True):
  ss_map = {'H':[1,0,0], 'E':[0,1,0], 'C':[0,0,1], '-':[0,0,1]}

  # check window is an odd integer
  assert type(window) == int, 'Error: window must be an integer!'
  assert window%2!=0, 'Error: window must be an ODD integer!'

  # select which data to fetch and load the right list of ids
  if not cv:
    path = '/content/protein-ss-pred/data/blindTest/'
    id_list = [line.rstrip() for line in open(path+'list.txt')]
    print ('You are getting the test data!')
  else:
    cv = str(cv)
    path = '/content/protein-ss-pred/data/training/'
    if train:
      id_list = [line.rstrip() for line in open(path+'/cv/train'+cv+'.txt')]
      print ('You are getting the train partition of cross-validation set '+cv)
    else:
      id_list = [line.rstrip() for line in open(path+'/cv/test'+cv+'.txt')]
      print ('You are getting the test partition of cross-validation set '+cv)

  X, Y = [], []
  for id in id_list:
    # fetch input features
    sequence, profile = np.array(parse_pssm(path+'/pssm/'+id+'.pssm'))
    x = np.concatenate((sequence, profile), axis=-1)

    # fetch and encode labels
    ss = parse_dssp(path+'/dssp/'+id+'.dssp')
    y = np.array([ss_map[c] for c in ss])

    #add (window-1)/2 padding on both sequence sides
    #to create window for first and last positions
    side = int((window-1)/2)
    x_pad = np.zeros((side, 40))
    x = np.concatenate((x_pad, x, x_pad), axis=0)

    #extract all windows
    X += [x[i-side:i+side+1,:] for i in range(side, len(x)-side-1)]
    Y += [y[i,:] for i in range(len(y))]
    X.append(x[-2*side-1:,:])

  return np.array(X), np.array(Y)

# Format some data

In [ ]:
window = 11
x_train_cv1, y_train_cv1 = fetch_data(window, cv=1)
print (x_train_cv1.shape, y_train_cv1.shape)

### the resulting input features array shape has:
# in the first position all amino acids in the selected train or test set
# in the second position the window size
# in the third position the number of features (amino acid identity and pssm frequencies)

### the resulting output label array shape has instead:
# in the first position all amino acids in the selected train or test set
# in the second position the number of classes

You are getting the train partition of cross-validation set 1
(157415, 11, 40) (157415, 3)


# Declare and train a neural network

In [ ]:
x_train_cv1 = np.array(x_train_cv1)
y_train_cv1 = np.array(y_train_cv1)
input_shape = (window, 40)
num_classes = 3

model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Conv1D(16, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

h = model.fit(x_train_cv1, y_train_cv1, batch_size=512, epochs=10, validation_split=0.1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 11, 16)            1936      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 5, 16)             0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 80)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2592      
                                                                 
 dense_3 (Dense)             (None, 3)                 99        
                                                                 
Total params: 4627 (18.07 KB)
Trainable params: 4627 (18.07 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

# Fetch test data, use fitted model to get predictions, and evaluate predictions

In [ ]:
x_test_cv1, y_test_cv1 = fetch_data(window, cv=1, train=False)

You are getting the test partition of cross-validation set 1


In [ ]:
predictions = model.predict(x_test_cv1)

1273/1273 [==============================] - 3s 2ms/step


In [ ]:
acc = 0
for y, y_real in zip(predictions, y_test_cv1):
  if y_real[np.argmax(y)] == 1:
    acc += 1
acc /= len(predictions)
print ('Accuracy:', round(acc,2))

Accuracy: 0.74


# Prediction for a single sequence (PSSM or id)

In [ ]:
def predict_secondary_structure(pssm_filepath, model, window_size):
    sequence_encoding, profile = parse_pssm(pssm_filepath)
    x = np.concatenate((sequence_encoding, profile), axis=-1)
    side = int((window_size - 1) / 2)
    x_pad = np.zeros((side, x.shape[1]))
    x_padded = np.concatenate((x_pad, x, x_pad), axis=0)
    windows = [x_padded[i-side:i+side+1, :] for i in range(side, len(x_padded)-side)]
    windows = np.array(windows)
    predictions = model.predict(windows)
    predicted_classes = np.argmax(predictions, axis=1)

    # Single element extraction, otherwise it creates an error
    mode_result = stats.mode(predicted_classes)
    most_common_class = mode_result.mode.item()
    class_to_label = {0: 'H', 1: 'E', 2: 'C'}
    predicted_structure = class_to_label.get(most_common_class, "Unknown")
    return predicted_structure

pssm_filepath = '/content/protein-ss-pred/data/blindTest/pssm/4S1H:A.pssm'
window_size = 11
predicted_structure = predict_secondary_structure(pssm_filepath, model, window_size)
print(f"Predicted Secondary Structure: {predicted_structure}")

9/9 [==============================] - 0s 3ms/step
Predicted Secondary Structure: C


In [ ]:
def predict_secondary_structure(id, model, window_size):
    sequence_encoding, profile = np.array(parse_pssm('/content/protein-ss-pred/data/blindTest/pssm/'+id+'.pssm'))
    x = np.concatenate((sequence_encoding, profile), axis=-1)
    side = int((window_size - 1) / 2)
    x_pad = np.zeros((side, x.shape[1]))
    x_padded = np.concatenate((x_pad, x, x_pad), axis=0)
    windows = [x_padded[i-side:i+side+1, :] for i in range(side, len(x_padded)-side)]
    windows = np.array(windows)
    predictions = model.predict(windows)
    predicted_classes = np.argmax(predictions, axis=1)

    # Single element extraction, otherwise it creates an error
    mode_result = stats.mode(predicted_classes)
    most_common_class = mode_result.mode.item()
    class_to_label = {0: 'H', 1: 'E', 2: 'C'}
    predicted_structure = class_to_label.get(most_common_class, "Unknown")
    return predicted_structure

id = '4S1H:A'
window_size = 11
predicted_structure = predict_secondary_structure(id, model, window_size)
print(f"Predicted Secondary Structure: {predicted_structure}")

9/9 [==============================] - 0s 4ms/step
Predicted Secondary Structure: C
